# Training data

In [51]:
import pickle
import numpy as np
import random
from operator import add

In [52]:
f = open ('./char_embedding.pkl', 'rb')
dict_char = pickle.load(f)
f.close()

f = open ('./dataset/item_index_withEN.pkl', 'rb')
item_index = pickle.load(f)
f.close()

f = open ('./dataset/name_index_withEN.pkl', 'rb')
user_index = pickle.load(f)
f.close()

In [53]:
rating_mat = np.load('dataset/user_item_withEN_mat.npy')

In [54]:
print("dict_char", len(dict_char))
print("user_index", len(user_index))
print("item_index", len(item_index))
print("rating_mat", rating_mat.shape)

dict_char 21592
user_index 15251
item_index 32
rating_mat (15251, 32)


In [55]:
#dict_char['沈']
#user_list.remove('')
k, v = random.choice(list(dict_char.items()))
print(type(v))
#print(user_list)

<class 'numpy.ndarray'>


In [59]:
user_list = list(user_index.keys())
print(len(user_list))
user_mat = []
user_con_mat = []
user_emb = []
con_emb = []
count = 0

for user in user_list:
    temp = []
    for char in user:
        try:
            temp.append(dict_char[char])
        except:
            count += 1
            k, v = random.choice(list(dict_char.items()))
            temp.append(v)
    #print(len(temp))
    if len(temp) == 2:
        # k, v = random.choice(list(dict_char.items()))
        user_emb = np.mean([temp[0], temp[1]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[1]))
    elif len(temp) == 3:
        user_emb = np.mean([temp[0], temp[1], temp[2]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[2]))
    elif len(temp) == 4:
        user_emb = np.mean([temp[0], temp[1], temp[2], temp[-1]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[2]))
    else:
        print("index", user_list.index(user))
        #rating_mat = np.delete(rating_mat, [user_list.index(user)], axis=0)
        user_list.remove(user)
        del user_index[user]
        print(len(temp))
    user_con_mat.append(con_emb)
    user_mat.append(user_emb)

user_mat = np.array(user_mat)
user_con_mat = np.array(user_con_mat)
print("missing char", count)
print("user_mat", user_mat.shape)
print("user_con_mat", user_con_mat.shape)
print("remain", len(user_list))
print("remain", rating_mat.shape)

15250
missing char 190
user_mat (15250, 256)
user_con_mat (15250, 768)
remain 15250
remain (15250, 32)


In [60]:
#item_shape = (256, 32)
#item_mat = np.random.rand(32, 256)
print(user_mat)

[[-0.25516173  0.02200902 -0.6367982  ...  0.27963406  0.06052778
  -0.2570803 ]
 [-0.00804286 -0.43483147 -0.4136071  ...  0.5910477   0.11055978
  -0.76872826]
 [-0.18077803 -0.28984937 -0.42697546 ...  0.38626578  0.0096873
  -0.5621365 ]
 ...
 [ 0.31248116 -0.27425033  0.06705994 ...  0.5632503   0.6493718
  -0.25115702]
 [-0.59236026  0.25725064  0.44351003 ...  0.8128228  -0.12497162
  -0.08346231]
 [-0.06863866  0.05283417  0.21988375 ...  0.6965713   0.10992792
  -0.58061045]]


In [61]:
num_user = 15250
num_item = 32

# Model

In [62]:
from keras import Model
import keras.backend as K
from keras.layers import Embedding,Reshape,Input,Dot, Dense, dot, Lambda
K.clear_session()

In [63]:
# def create_model(num_user, num_item, k):
#     input_user = Input(shape=(256,),dtype="float32")
#     #model_uer = Embedding(num_user+1,k,input_length = 1)(input_uer)
#     model_user = Dense(k, activation='relu')(input_user)
#     model_user = Reshape((k,))(model_user)
    
#     input_item = Input(shape=(32,),dtype="float32")
#     model_item  = Embedding(num_item+1,k,input_length = 32)(input_item)
#     model_item = Dense(256, activation='relu')(model_item)
#     model_item = Reshape((256,))(model_item)
    
#     out = Dot(1)([model_user,model_item])
#     model = Model(inputs=[input_user,model_item], outputs=out)
#     model.compile(loss='mse', optimizer='Adam')
#     model.summary()
#     return model

In [74]:
def Recmand_model(num_user,num_item,k):
    input_uer = Input(shape=(768, ),dtype="float32")
    model_uer = Dense(256, activation='relu')(input_uer)
    model_uer = Dense(128, activation='relu')(model_uer)
    model_uer = Dense(k, activation='relu')(model_uer)
    
    
    input_item = Input(shape=(1,), dtype="float32")
    model_item  = Embedding(num_item, k, input_length = 1)(input_item)
    model_item = Reshape((k,))(model_item)
    
    out = Lambda(lambda x: K.sum(x[0]*x[1], axis=-1, keepdims=True))([model_uer, model_item])
    model = Model(inputs=[input_uer, input_item], outputs=out)
    model.compile(loss='mse', optimizer='Adam')
    model.summary()
    return model

In [75]:
model = Recmand_model(num_user, num_item, 64)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 768)          0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 256)          196864      input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 128)          32896       dense_2[0][0]                    
__________________________________________________________________________________________________
embedding_

In [76]:
# batch_size = 10
test_user = np.random.random((10, 256))
test_item = np.random.randint(0,32, (10))

In [77]:
print(test_user)

[[0.77728389 0.77301389 0.98034433 ... 0.41183877 0.96137171 0.06833239]
 [0.34094176 0.52559156 0.95608407 ... 0.56940886 0.73925566 0.0494454 ]
 [0.96771409 0.84966125 0.96773883 ... 0.27405069 0.16022994 0.79279835]
 ...
 [0.51825847 0.53417301 0.26936317 ... 0.09125106 0.49868644 0.11826715]
 [0.77286229 0.05623031 0.68284393 ... 0.11435994 0.54097476 0.52244329]
 [0.13613644 0.56716673 0.6145515  ... 0.54017357 0.23930877 0.48211563]]


In [78]:
print(test_item)

[22 30 26  8 12 22 12 14  3 31]


In [79]:
# test predict
#model.predict([np.random.random((10, 256)), np.random.randint(0,32, (10))])

In [80]:
print(user_con_mat)

[[-0.74557173 -0.02800325 -0.72160846 ...  0.50239176  0.42607114
  -0.61805236]
 [-0.00291761 -0.34882382 -0.24279383 ...  0.7806603  -0.12670909
  -1.0558528 ]
 [-0.4240876  -0.30831465  0.5773001  ...  0.01698863  0.14114247
  -0.6353096 ]
 ...
 [-0.02675579 -0.8901479   0.57638884 ...  0.35549855  0.82068586
   0.22624786]
 [-0.48385873  0.4444883   0.65895814 ...  1.3389612  -0.20171902
  -0.702008  ]
 [ 0.01715979  0.4324117   0.3835889  ...  0.98461074  0.3981392
  -0.48753178]]


In [81]:
user_index = 0
item_index = 0
train_x = []
train_user = []
train_item = []

for u in rating_mat:
    for i in u:
        if i == 1:
            # train_x.append([user_mat[user_index], [item_index]])
            u_list = user_con_mat[user_index].tolist()
            train_user.append(u_list)
            #print(user_mat[user_index].tolist())
            train_item.append(item_index)
        item_index += 1
    user_index += 1
    item_index = 0
train_x = [np.array(train_user), np.array(train_item)]

In [82]:
#train_x = np.array(train_x)
print(train_x)
#print(train_x)
train_y = [1] * 39911
train = [1] * 10
#print(train_y)

[array([[-0.74557173, -0.02800325, -0.72160846, ...,  0.50239176,
         0.42607114, -0.61805236],
       [-0.74557173, -0.02800325, -0.72160846, ...,  0.50239176,
         0.42607114, -0.61805236],
       [-0.74557173, -0.02800325, -0.72160846, ...,  0.50239176,
         0.42607114, -0.61805236],
       ...,
       [-0.02675579, -0.89014792,  0.57638884, ...,  0.35549855,
         0.82068586,  0.22624786],
       [-0.48385873,  0.44448829,  0.65895814, ...,  1.33896124,
        -0.20171902, -0.70200801],
       [ 0.01715979,  0.4324117 ,  0.38358891, ...,  0.98461074,
         0.39813921, -0.48753178]]), array([6, 7, 8, ..., 0, 0, 0])]


In [83]:
model.fit(train_x, train_y, batch_size = 64, epochs =300, verbose=1, validation_split=0.2, shuffle=True)

Train on 31928 samples, validate on 7983 samples
Epoch 1/300
31928/31928 [==============================] - 3s 81us/step - loss: 0.0128 - val_loss: 0.1897
Epoch 2/300
31928/31928 [==============================] - 2s 70us/step - loss: 1.9164e-04 - val_loss: 0.1911
Epoch 3/300
31928/31928 [==============================] - 2s 70us/step - loss: 1.8124e-04 - val_loss: 0.1926
Epoch 4/300
31928/31928 [==============================] - 2s 70us/step - loss: 2.7323e-04 - val_loss: 0.1911
Epoch 5/300
31928/31928 [==============================] - 2s 70us/step - loss: 1.4453e-04 - val_loss: 0.1919
Epoch 6/300
31928/31928 [==============================] - 2s 70us/step - loss: 2.9616e-04 - val_loss: 0.1933
Epoch 7/300
31928/31928 [==============================] - 2s 72us/step - loss: 1.0198e-04 - val_loss: 0.1936
Epoch 8/300
31928/31928 [==============================] - 2s 72us/step - loss: 2.3882e-04 - val_loss: 0.1938
Epoch 9/300
31928/31928 [==============================] - 2s 71us/step - l

31928/31928 [==============================] - 2s 71us/step - loss: 3.6077e-07 - val_loss: 0.1986
Epoch 75/300
31928/31928 [==============================] - 2s 71us/step - loss: 3.5745e-06 - val_loss: 0.1986
Epoch 76/300
31928/31928 [==============================] - 2s 73us/step - loss: 3.1813e-05 - val_loss: 0.1985
Epoch 77/300
31928/31928 [==============================] - 2s 74us/step - loss: 1.6445e-06 - val_loss: 0.1985
Epoch 78/300
31928/31928 [==============================] - 3s 79us/step - loss: 2.5521e-05 - val_loss: 0.1984
Epoch 79/300
31928/31928 [==============================] - 2s 77us/step - loss: 4.3134e-06 - val_loss: 0.1984
Epoch 80/300
31928/31928 [==============================] - 2s 72us/step - loss: 1.5104e-05 - val_loss: 0.1983
Epoch 81/300
31928/31928 [==============================] - 2s 74us/step - loss: 2.0092e-05 - val_loss: 0.1984
Epoch 82/300
31928/31928 [==============================] - 3s 79us/step - loss: 3.3793e-06 - val_loss: 0.1984
Epoch 83/300
3

31928/31928 [==============================] - 2s 70us/step - loss: 1.4045e-05 - val_loss: 0.1980
Epoch 148/300
31928/31928 [==============================] - 2s 70us/step - loss: 7.7147e-06 - val_loss: 0.1980
Epoch 149/300
31928/31928 [==============================] - 2s 70us/step - loss: 4.0423e-06 - val_loss: 0.1979
Epoch 150/300
31928/31928 [==============================] - 2s 70us/step - loss: 1.2978e-05 - val_loss: 0.1980
Epoch 151/300
31928/31928 [==============================] - 2s 70us/step - loss: 2.6847e-06 - val_loss: 0.1979
Epoch 152/300
31928/31928 [==============================] - 2s 70us/step - loss: 1.0321e-05 - val_loss: 0.1979
Epoch 153/300
31928/31928 [==============================] - 2s 70us/step - loss: 4.8358e-06 - val_loss: 0.1981
Epoch 154/300
31928/31928 [==============================] - 2s 70us/step - loss: 1.0598e-05 - val_loss: 0.1981
Epoch 155/300
31928/31928 [==============================] - 2s 70us/step - loss: 4.0977e-06 - val_loss: 0.1981
Epoch 

31928/31928 [==============================] - 2s 70us/step - loss: 1.9009e-06 - val_loss: 0.1969
Epoch 221/300
31928/31928 [==============================] - 2s 70us/step - loss: 5.8299e-06 - val_loss: 0.1973
Epoch 222/300
31928/31928 [==============================] - 2s 70us/step - loss: 1.9594e-06 - val_loss: 0.1981
Epoch 223/300
31928/31928 [==============================] - 2s 70us/step - loss: 5.0997e-06 - val_loss: 0.1986
Epoch 224/300
31928/31928 [==============================] - 2s 69us/step - loss: 1.5447e-06 - val_loss: 0.1986
Epoch 225/300
31928/31928 [==============================] - 2s 70us/step - loss: 6.3964e-06 - val_loss: 0.1987
Epoch 226/300
31928/31928 [==============================] - 2s 70us/step - loss: 2.0200e-06 - val_loss: 0.1987
Epoch 227/300
31928/31928 [==============================] - 2s 71us/step - loss: 4.3991e-06 - val_loss: 0.1984
Epoch 228/300
31928/31928 [==============================] - 2s 70us/step - loss: 3.1915e-06 - val_loss: 0.1989
Epoch 

31928/31928 [==============================] - 2s 69us/step - loss: 9.1534e-07 - val_loss: 0.1983
Epoch 294/300
31928/31928 [==============================] - 2s 70us/step - loss: 2.8968e-06 - val_loss: 0.1983
Epoch 295/300
31928/31928 [==============================] - 2s 74us/step - loss: 2.3683e-06 - val_loss: 0.1984
Epoch 296/300
31928/31928 [==============================] - 2s 77us/step - loss: 1.2492e-06 - val_loss: 0.1984
Epoch 297/300
31928/31928 [==============================] - 2s 70us/step - loss: 2.1635e-06 - val_loss: 0.1984
Epoch 298/300
31928/31928 [==============================] - 2s 70us/step - loss: 2.9078e-06 - val_loss: 0.1984
Epoch 299/300
31928/31928 [==============================] - 2s 72us/step - loss: 3.6680e-07 - val_loss: 0.1984
Epoch 300/300
31928/31928 [==============================] - 3s 83us/step - loss: 3.5802e-06 - val_loss: 0.1986


In [84]:
model.save('my_model_con_300.h5')

In [86]:
#embeddings = model.layers[2].get_weights()[0]
embeddings = model.layers[4].get_weights()[0]

In [87]:
embeddings.shape
# embed = embeddings.T
# print(embed.shape)

(32, 64)

## Predict 

In [127]:
sc_name_dict = {'CGU':"長庚大學", 'CMU':"中國醫藥大學", 'CYCU':"中原大學", 'FCU':"逢甲大學", 'KMU':"高雄醫學大學", 
                'NCCU':"政治大學", 'NCHU':"中興大學", 'NCKU':"成功大學", 'NCTU':"交通大學", 'NCUE':"彰化師範大學", 
                'NDHU':"東華大學", 'NKNU':"高雄師範大學", 'NPTU':"屏東大學", 'NSYSU':"中山大學", 'NTCU':"臺中教育大學", 
                'NTHU':"清華大學", 'NTNU':"臺灣師範大學", 'NTU':"臺灣大學", 'NTUE':"臺北教育大學", 'NUTN':"臺南大學",
                'SCU':"東吳大學", 'THU':"東海大學", 'TKU':"淡江大學", 'TNUA':"臺北藝術大學", 'UTAIPEI':"臺北市立大學",
                'YM':"陽明大學"}

In [99]:
names = ["馬崇堯", "周東誼", "鬍子元", "龍澳天", "黃智賢"]
#name = "周東誼"

In [100]:
def getCharEmbed(name):
    temp = []
    for char in name:
        try:
            temp.append(dict_char[char])
        except:
            k, v = random.choice(list(dict_char.items()))
            temp.append(v)
            
    if len(temp) == 2:
        # k, v = random.choice(list(dict_char.items()))
        user_emb = np.mean([temp[0], temp[1]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[1]))
    elif len(temp) == 3:
        user_emb = np.mean([temp[0], temp[1], temp[2]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[2]))
    elif len(temp) == 4:
        user_emb = np.mean([temp[0], temp[1], temp[2], temp[-1]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[2]))
    else:
        for i in range(3):
            k, v = random.choice(list(dict_char.items()))
            temp.append(v)
        user_emb = v
        con_emb = np.concatenate((temp[0], temp[1], temp[2]))
    
    return con_emb

In [101]:
us = []
for name in names:
    u = getCharEmbed(name)
    us.append(u)
sc = np.array(range(32))

In [102]:
print(us[0].shape)

(768,)


In [92]:
#model.predict([np.random.random((10, 256)), np.random.randint(0,32, (10))])
#np.random.random((10, 256))

In [95]:
f = open ('./dataset/item_index_withEN.pkl', 'rb')
item_index_dict = pickle.load(f)
f.close()

item_index_dict 

{'董事': 0,
 '董事長': 1,
 '獨立董事': 2,
 '副董事長': 3,
 '常務董事': 4,
 '執行業務股東': 5,
 'CGU': 6,
 'CMU': 7,
 'CYCU': 8,
 'FCU': 9,
 'KMU': 10,
 'NCCU': 11,
 'NCHU': 12,
 'NCKU': 13,
 'NCTU': 14,
 'NCUE': 15,
 'NDHU': 16,
 'NKNU': 17,
 'NPTU': 18,
 'NSYSU': 19,
 'NTCU': 20,
 'NTHU': 21,
 'NTNU': 22,
 'NTU': 23,
 'NTUE': 24,
 'NUTN': 25,
 'SCU': 26,
 'THU': 27,
 'TKU': 28,
 'TNUA': 29,
 'UTAIPEI': 30,
 'YM': 31}

In [96]:
def getSchool(item_index_dict, score_list):
    broad_list = []
    for i in range(len(score_list)):
        if score_list[i] == 1:
            for k, v in item_index_dict.items():
                if v == i:
                    broad_list.append(k)
    return broad_list

In [108]:
for i in range(len(us)):
    print(names[i])
    u = us[i]
    score_list = []
    for i in range(32):
        output = model.predict([[u], [i]])
        if output[0][0] > 0.85:
            score_list.append(1)
        else:
            score_list.append(0)
    print(score_list)
    output_list = getSchool(item_index_dict, score_list)

馬崇堯
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0]
['CGU', 'FCU', 'KMU', 'NCCU', 'NCHU', 'NCTU', 'NDHU', 'NPTU', 'NSYSU', 'NTCU', 'NTU', 'NTUE', 'NUTN', 'THU', 'TNUA', 'UTAIPEI']
周東誼
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[]
鬍子元
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[]
龍澳天
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['CGU', 'CMU', 'CYCU', 'FCU', 'KMU', 'NCCU', 'NCHU', 'NCKU', 'NCTU', 'NCUE', 'NDHU', 'NKNU', 'NPTU', 'NSYSU', 'NTCU', 'NTHU', 'NTNU', 'NTU', 'NTUE', 'NUTN', 'SCU', 'THU', 'TKU', 'TNUA', 'UTAIPEI', 'YM']
黃智賢
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
['FCU', 'NSYSU', 'NTCU', 'NTU']


In [94]:
#print(score_list)

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [97]:
output_list = getSchool(item_index_dict, score_list)

In [98]:
print(output_list)
print(len(output_list))

['CGU', 'CMU', 'CYCU', 'FCU', 'KMU', 'NCCU', 'NCHU', 'NCKU', 'NCTU', 'NCUE', 'NDHU', 'NKNU', 'NPTU', 'NSYSU', 'NTCU', 'NTHU', 'NTNU', 'NTU', 'NTUE', 'NUTN', 'SCU', 'THU', 'TKU', 'TNUA', 'UTAIPEI', 'YM']
26


In [167]:
def predictSchool(name):
    count = 0
    max_score = 0
    max_sc_index = 0
    print(name)
    u = getCharEmbed(name)
    score_list = []
    for i in range(32):
        output = model.predict([[u], [i]])
        if output[0][0] > max_score:
            max_score = output[0][0]
            max_sc_index = i
        if output[0][0] > 0.85:
            score_list.append(1)
            count += 1
        else:
            score_list.append(0)
    print("你會上的學校或職位有 :", count, "個")
    output_list = getSchool(item_index_dict, score_list)
    print("你會上的學校或職位有 :")
    for out in output_list:
        print(sc_name_dict[out], end=", ")
    if count > 0:
        print("\n機率最大的是:", list(sc_name_dict.values())[max_sc_index-6])
        #print(max_sc_index)
    print("\n")
    max_sc_index = 0
    

In [168]:
for name in names:
    predictSchool(name)

馬崇堯
你會上的學校或職位有 : 16 個
你會上的學校或職位有 :
長庚大學, 逢甲大學, 高雄醫學大學, 政治大學, 中興大學, 交通大學, 東華大學, 屏東大學, 中山大學, 臺中教育大學, 臺灣大學, 臺北教育大學, 臺南大學, 東海大學, 臺北藝術大學, 臺北市立大學, 
機率最大的是: 臺南大學


周東誼
你會上的學校或職位有 : 0 個
你會上的學校或職位有 :


鬍子元
你會上的學校或職位有 : 0 個
你會上的學校或職位有 :


龍澳天
你會上的學校或職位有 : 26 個
你會上的學校或職位有 :
長庚大學, 中國醫藥大學, 中原大學, 逢甲大學, 高雄醫學大學, 政治大學, 中興大學, 成功大學, 交通大學, 彰化師範大學, 東華大學, 高雄師範大學, 屏東大學, 中山大學, 臺中教育大學, 清華大學, 臺灣師範大學, 臺灣大學, 臺北教育大學, 臺南大學, 東吳大學, 東海大學, 淡江大學, 臺北藝術大學, 臺北市立大學, 陽明大學, 
機率最大的是: 逢甲大學


黃智賢
你會上的學校或職位有 : 4 個
你會上的學校或職位有 :
逢甲大學, 中山大學, 臺中教育大學, 臺灣大學, 
機率最大的是: 中山大學




In [169]:
predictSchool("韓國瑜")

韓國瑜
你會上的學校或職位有 : 13 個
你會上的學校或職位有 :
長庚大學, 逢甲大學, 政治大學, 交通大學, 東華大學, 高雄師範大學, 屏東大學, 臺中教育大學, 臺灣大學, 臺南大學, 東海大學, 臺北藝術大學, 臺北市立大學, 
機率最大的是: 臺南大學




In [170]:
predictSchool("黃崇明")

黃崇明
你會上的學校或職位有 : 6 個
你會上的學校或職位有 :
逢甲大學, 臺中教育大學, 臺灣大學, 臺南大學, 東海大學, 臺北藝術大學, 
機率最大的是: 臺南大學




In [171]:
predictSchool("王麒詳")

王麒詳
你會上的學校或職位有 : 26 個
你會上的學校或職位有 :
長庚大學, 中國醫藥大學, 中原大學, 逢甲大學, 高雄醫學大學, 政治大學, 中興大學, 成功大學, 交通大學, 彰化師範大學, 東華大學, 高雄師範大學, 屏東大學, 中山大學, 臺中教育大學, 清華大學, 臺灣師範大學, 臺灣大學, 臺北教育大學, 臺南大學, 東吳大學, 東海大學, 淡江大學, 臺北藝術大學, 臺北市立大學, 陽明大學, 
機率最大的是: 中山大學




In [172]:
predictSchool("趙偉廷")

趙偉廷
你會上的學校或職位有 : 0 個
你會上的學校或職位有 :


